# Merge CTD and satellite data extracted at observation locations
Created by Ivan Lima on Tue Nov  2 2021 14:42:35 -0400

In [1]:
%matplotlib inline
import pandas as pd
import xarray as xr
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime, warnings, os
print('Last updated on {}'.format(datetime.datetime.now().ctime()))

Last updated on Mon Dec  6 11:42:29 2021


In [2]:
sns.set_theme(context='paper', style='ticks', palette='tab10', rc={'figure.dpi':100, 'figure.figsize':[5, 5], 'axes.grid':True})
pd.options.display.max_columns = 50
warnings.filterwarnings('ignore')

## Read satellite & CTD data

In [3]:
# Read & concatenate annual satellite data files into one
datadir = 'data/by_year'
filelist = sorted([os.path.join(datadir, file) for file in os.listdir('data/by_year/')])
df_list = []
for file in filelist:
    df_list.append(pd.read_csv(file, parse_dates=['Date'], index_col=0))
    
df_sat = pd.concat(df_list, axis=0)
df_sat.head()

,Cast,Latitude,Longitude,Year,Month,Day,Platform_Type,bottom_depth,Date,SLA,SST,SST_hires,Chl,KD490
52697,804172,44.083332,-63.000000,1981,1,6,3,211,1981-01-06,NaN,NaN,NaN,NaN,NaN
52698,804175,43.766666,-62.983334,1981,1,6,3,265,1981-01-06,NaN,NaN,NaN,NaN,NaN
52699,804165,44.066666,-63.183334,1981,1,6,3,160,1981-01-06,NaN,NaN,NaN,NaN,NaN
52700,804170,43.950001,-62.916668,1981,1,6,3,256,1981-01-06,NaN,NaN,NaN,NaN,NaN
52701,804181,43.983334,-63.099998,1981,1,7,3,184,1981-01-07,NaN,NaN,NaN,NaN,NaN


In [4]:
ds_ctd = xr.open_dataset('data/CombinedCTD.nc')
df_ctd = ds_ctd.to_dataframe()
df_ctd = df_ctd.droplevel(0) # drop "column" index
df_ctd.index.name = None
df_ctd = df_ctd[df_ctd.Date_YYYYMMDD!=19220100] # remove invalid dates
df_ctd['Date'] = pd.to_datetime(df_ctd['Date_YYYYMMDD'], format='%Y%m%d')
df_ctd = df_ctd.drop(['Date_YYYYMMDD','Time_DaysSince1_1_1770'],axis=1)
df_ctd = df_ctd[df_ctd.Date.dt.year > 1980] # only data from satellite era
col_names = {'CastID':'Cast', 'Lat':'Latitude', 'Lon':'Longitude', 'Depth_m':'Depth',
             'Temp_C':'Temperature', 'Sal_PSU':'Salinity', 'DO_umolkg':'Oxygen',
             'Temp_Flag':'Temp_flag', 'Sal_Flag':'Salt_flag'}
df_ctd = df_ctd.rename(columns=col_names) # rename columns

In [5]:
df_ctd.head()

,Cast,Latitude,Longitude,Depth,Temperature,Temp_flag,Salinity,Salt_flag,Oxygen,DO_flag,Platform_Type,pCO2_year,pCO2_month,Date
864823,3344598.0,41.450001,-64.150002,19.260000,NaN,NaN,32.700001,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864824,3344598.0,41.450001,-64.150002,20.959999,NaN,NaN,32.700001,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864825,3344598.0,41.450001,-64.150002,23.230000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864826,3344598.0,41.450001,-64.150002,24.850000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05
864827,3344598.0,41.450001,-64.150002,26.830000,NaN,NaN,32.701000,0.0,NaN,NaN,1.0,340.12,341.63,1981-03-05


## Do inner join 

In [6]:
# sat_cols = ['Cast', 'Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Date', 'ADT',
#             'UGOS', 'VGOS', 'SLA', 'UGOSA', 'VGOSA', 'SST', 'SST_hires', 'Chl', 'KD490']

# ctd_cols = ['X', 'Cast', 'Latitude', 'Longitude', 'Year', 'Month', 'Day', 'Date', 'Depth', 'Temperature', 'Salinity',
#             'Dissolved.oxygen', 'Pressure', 'pCO2_yearave', 'pCO2_monthave', 'Lat', 'Long', 'bottom_depth',
#             'lat', 'long', 'Oxygen', 'OmegaA','OmegaC', 'CO3', 'fCO2', 'pCO2', 'pH', 'HCO3']

sat_cols = ['Cast', 'Latitude', 'Longitude', 'Date', 'bottom_depth', 'SLA', 'SST', 'SST_hires', 'Chl', 'KD490']

ctd_cols = ['Cast', 'Latitude', 'Longitude', 'Date', 'Depth', 'Temperature', 'Salinity', 'Oxygen',
            'pCO2_year', 'pCO2_month', 'Platform_Type', 'Temp_flag', 'Salt_flag', 'DO_flag']

# df_merged = pd.merge(df_ctd[ctd_cols], df_sat[sat_cols], on=['Date','Latitude','Longitude'],
#                      how='inner', suffixes=['_ctd', '_sat'])
df_merged = pd.merge(df_ctd[ctd_cols], df_sat[sat_cols], on=['Cast'],
                     how='inner', suffixes=['_ctd', '_sat'])

print('Original data: {:,} rows'.format(len(df_ctd)))
print('New merged data: {:,} rows'.format(len(df_merged)))
print('Difference: {:,} rows'.format(len(df_merged)-len(df_ctd)))

Original data: 26,330,017 rows
New merged data: 14,539,802 rows
Difference: -11,790,215 rows


## Check if records match for merged variables 

In [7]:
def check(vname):
    vname_ctd, vname_sat = vname+'_ctd', vname+'_sat'
    df_work = df_merged[[vname_ctd, vname_sat]].dropna()
    if len(df_work) == (df_work[vname_ctd]==df_work[vname_sat]).sum():
        print('{}: OK'.format(vname))
    else:
        n = len(df_work) - (df_work[vname_ctd]==df_work[vname_sat]).sum()
        print('{}: {:,} records do not match'.format(vname, n))

varcols = [c[:-4] for c in df_merged.columns if c.endswith('_ctd')]
for vname in varcols:
    check(vname)

Latitude: 13,039,876 records do not match
Longitude: 11,775,196 records do not match
Date: OK


**Note:** The *Date* matches for all records but *Latitude* and *Longitude* don't due to differences in precision.

In [8]:
df_merged[['Date_ctd','Date_sat','Latitude_ctd','Latitude_sat','Longitude_ctd','Longitude_sat']].head()

,Date_ctd,Date_sat,Latitude_ctd,Latitude_sat,Longitude_ctd,Longitude_sat
0,1981-03-10,1981-03-10,42.25,42.25,-65.466698,-65.466698
1,1981-03-10,1981-03-10,42.25,42.25,-65.466698,-65.466698
2,1981-03-10,1981-03-10,42.25,42.25,-65.466698,-65.466698
3,1981-03-10,1981-03-10,42.25,42.25,-65.466698,-65.466698
4,1981-03-10,1981-03-10,42.25,42.25,-65.466698,-65.466698


In [9]:
lat_diff  = df_merged.Latitude_ctd - df_merged.Latitude_sat
lat_diff.min(), lat_diff.max()

(-4.973799150320701e-14, 4.973799150320701e-14)

In [10]:
lon_diff  = df_merged.Longitude_ctd - df_merged.Longitude_sat
lon_diff.min(), lon_diff.max()

(-4.973799150320701e-14, 5.684341886080802e-14)

## Save merged data set to HDF file 

In [11]:
df_merged_final = df_merged.drop(['Latitude_sat','Longitude_sat','Date_sat'], axis=1).rename(columns={'Latitude_ctd':'Latitude',
                                                                                                        'Longitude_ctd':'Longitude',
                                                                                                        'Date_ctd':'Date'})
yr_min, yr_max = df_merged_final.Date.dt.year.min(), df_merged_final.Date.dt.year.max()
outfile = 'data/CombinedCTD_satellite_{}-{}.h5'.format(yr_min, yr_max)
print('writing {}'.format(outfile))
df_merged_final.to_hdf(outfile, key='df_ctd_sat', mode='w', complevel=9)

writing data/CombinedCTD_satellite_1981-2021.h5
